# Analysis of  Spaceship Titanic Data

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spaceship_Transportation_Analytics").getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark

22/10/04 20:55:41 WARN Utils: Your hostname, adnan-System-Product-Name resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface enp2s0)
22/10/04 20:55:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 20:55:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df= spark.read.csv("spaceship/train.csv",header=True,inferSchema=True)

In [3]:
import pyspark.pandas as ps

psdf = df.toPandas().set_index("PassengerId")
psdf.head(5)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [15]:
df.crosstab("VIP","Transported").show()

+---------------+-----+----+
|VIP_Transported|false|true|
+---------------+-----+----+
|           true|  123|  76|
|           null|   99| 104|
|          false| 4093|4198|
+---------------+-----+----+

22/10/04 22:40:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3696212 ms exceeds timeout 120000 ms
22/10/04 22:40:14 WARN SparkContext: Killing executors is not supported by current scheduler.


In [14]:
psdf["HomePlanet"].unique()

array(['Europa', 'Earth', 'Mars', None], dtype=object)